# Raster index

In [1]:
%xmode minimal

import xarray as xr

import rasterix

xr.set_options(display_expand_indexes=True)

Exception reporting mode: Minimal


## Example with rectilinear and no rotation affine transform

Both x and y coordinates are 1-dimensional.

In [2]:
source = "/vsicurl/https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"

### Load and inspect the datasets, with and without `RasterIndex`.

In [3]:
da_no_raster_index = xr.open_dataarray(source, engine="rasterio")
da_raster_index = rasterix.assign_index(da_no_raster_index)

Let's compare the coordinates for the DataArrays with and without RasterIndex

In [4]:
da_no_raster_index.x

<xarray.DataArray 'x' (x: 316)> Size: 3kB
array([-3937500., -3912500., -3887500., ...,  3887500.,  3912500.,  3937500.],
      shape=(316,))
Coordinates:
  * x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
    spatial_ref  int64 8B ...

By contrast, these are lazy!

In [5]:
da_raster_index.x

<xarray.DataArray 'x' (x: 316)> Size: 3kB
[316 values with dtype=float64]
Coordinates:
    spatial_ref  int64 8B ...
    x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06

These differences are viewable in the repr. Note the `PandasIndex` type under "Indexes".

In [6]:
da_no_raster_index

<xarray.DataArray 'band_data' (band: 1, y: 332, x: 316)> Size: 420kB
[104912 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area

 The repr below shows a few values for each coordinate (those have been computed on-the-fly) but clicking on the database icon doesn't show any value in the spatial coordinate data reprs.

In [7]:
da_raster_index

<xarray.DataArray 'band_data' (band: 1, y: 332, x: 316)> Size: 420kB
[104912 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

### Positional Indexing (`isel`)

#### Slicing both x and y

In [8]:
da_sliced = da_raster_index.isel(x=slice(1, 4), y=slice(None, None, 2))
da_sliced

<xarray.DataArray 'band_data' (band: 1, y: 166, x: 3)> Size: 2kB
[498 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (x) float64 24B -3.912e+06 -3.888e+06 -3.862e+06
  * y            (y) float64 1kB 4.338e+06 4.288e+06 ... -3.862e+06 -3.912e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

Slicing keeps both coordinates lazy (it computes a new affine transform):

In [9]:
print(da_sliced.xindexes["x"])
print(da_sliced.xindexes["y"])

RasterIndex
'x':
'y':
RasterIndex
'x':
'y':


#### Outer indexing with arbitrary array values*

In [10]:
da_outer = da_raster_index.isel(x=[0, 2, 4], y=[0, 0, 1])
da_outer

<xarray.DataArray 'band_data' (band: 1, y: 3, x: 3)> Size: 36B
[9 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (x) float64 24B -3.938e+06 -3.888e+06 -3.838e+06
  * y            (y) float64 24B 4.338e+06 4.338e+06 4.312e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

We cannot compute a new affine transform given arbitrary array positions. To allow further data selection, pandas indexes are _automatically_ created for indexed spatial coordinates:

In [11]:
print(da_outer.xindexes["x"])
print(da_outer.xindexes["y"])

RasterIndex
'x':
    PandasIndex(Index([-3937500.0, -3887500.0, -3837500.0], dtype='float64', name='x'))
'y':
    PandasIndex(Index([4337500.0, 4337500.0, 4312500.0], dtype='float64', name='y'))
RasterIndex
'x':
    PandasIndex(Index([-3937500.0, -3887500.0, -3837500.0], dtype='float64', name='x'))
'y':
    PandasIndex(Index([4337500.0, 4337500.0, 4312500.0], dtype='float64', name='y'))


#### Basic indexing with scalars

In [12]:
da_scalar = da_raster_index.isel(x=0, y=1)
da_scalar

<xarray.DataArray 'band_data' (band: 1)> Size: 4B
[1 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
    x            float64 8B -3.938e+06
    y            float64 8B 4.312e+06
Attributes:
    AREA_OR_POINT:  Area

In [13]:
da_xscalar = da_raster_index.isel(x=0)
da_xscalar

<xarray.DataArray 'band_data' (band: 1, y: 332)> Size: 1kB
[332 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
    x            float64 8B -3.938e+06
    y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
Attributes:
    AREA_OR_POINT:  Area

**FIXME** The RasterIndex should be preserved in case of partial dimension reduction.

In [14]:
# da_xscalar.xindexes["y"]  # should return an index

#### Vectorized (fancy) indexing

Indexing the spatial coordinates with Xarray `Variable` objects returns a `RasterIndex` (wrapping `PandasIndex`) for 1-dimensional variables and no index for scalar or n-dimensional variables.

In [15]:
da_points = da_raster_index.isel(x=xr.Variable("z", [0, 1]), y=xr.Variable("z", [1, 1]))
da_points

<xarray.DataArray 'band_data' (band: 1, z: 2)> Size: 8B
[2 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (z) float64 16B -3.938e+06 -3.912e+06
  * y            (z) float64 16B 4.312e+06 4.312e+06
Dimensions without coordinates: z
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

In [16]:
da_points2d = da_raster_index.isel(
    x=xr.Variable(("u", "v"), [[0, 1], [2, 3]]),
    y=xr.Variable(("u", "v"), [[1, 1], [2, 2]]),
)
da_points2d

<xarray.DataArray 'band_data' (band: 1, u: 2, v: 2)> Size: 16B
[4 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
    x            (u, v) float64 32B -3.938e+06 -3.912e+06 -3.888e+06 -3.862e+06
    y            (u, v) float64 32B 4.312e+06 4.312e+06 4.288e+06 4.288e+06
Dimensions without coordinates: u, v
Attributes:
    AREA_OR_POINT:  Area

### Label Indexing (`sel`)

Label-based indexing also preserves the RasterIndex where possible.

In [17]:
da_raster_index.sel(x=slice(-2e6, 2e6), y=slice(4e6, -2e6))

<xarray.DataArray 'band_data' (band: 1, y: 240, x: 160)> Size: 154kB
[38400 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (x) float64 1kB -1.988e+06 -1.962e+06 ... 1.962e+06 1.988e+06
  * y            (y) float64 2kB 3.988e+06 3.962e+06 ... -1.962e+06 -1.988e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

### Equality

`equals` compares variable values without relying on Xarray coordinate indexes. Both dataarrays should thus be equal.

In [18]:
da_raster_index.equals(da_no_raster_index)

True

### Alignment

Xarray alignment relies on Xarray coordinate indexes. Trying to align both datasets fails here since they each have different index types.

In [19]:
da_raster_index + da_no_raster_index

ValueError: cannot re-index or align objects with conflicting indexes found for the following coordinates: 'y' (2 conflicting indexes), 'x' (2 conflicting indexes)
Conflicting indexes may occur when
- they relate to different sets of coordinate and/or dimension names
- they don't have the same type
- they may be used to reindex data along common dimensions

Implementing joins is on the roadmap.

In [20]:
xr.align(da_raster_index, da_raster_index.isel(x=slice(10), y=slice(10)), join="inner")

NotImplementedError: RasterIndex
'x':
    <rasterix.raster_index.AxisAffineTransformIndex object at 0x10ecdb5f0>
'y':
    <rasterix.raster_index.AxisAffineTransformIndex object at 0x10ecdb5c0> doesn't support alignment with inner/outer join method